In [ ]:
import datetime
import json
from time import sleep
from confluent_kafka import Producer
import pandas as pd



In [ ]:
def read_ccloud_config(config_file):
    omitted_fields = set(['schema.registry.url', 'basic.auth.credentials.source', 'basic.auth.user.info'])
    conf = {}
    with open(config_file) as fh:
        for line in fh:
            line = line.strip()
            if len(line) != 0 and line[0] != "#":
                parameter, value = line.strip().split('=', 1)
                if parameter not in omitted_fields:
                    conf[parameter] = value.strip()
    return conf


producer = Producer(read_ccloud_config("client.properties"))

In [ ]:

# topic schema

schema = {
  "doc": "Sample schema to help you get started.",
  "fields": [
    {
      "doc": "The int type is a 32-bit signed integer.",
      "name": "ppg",
      "type": "int"
    },
    {
      "doc": "timer timestamp.",
      "logicalType": "timestamp-micros",
      "name": "timer",
      "type": "string"
    },
    {
      "doc": "ID",
      "name": "index",
      "type": "int"
    }
  ],
  "name": "sampleRecord_heartrate",
  "namespace": "sample_heart_rate",
  "type": "record"
}

In [ ]:
DATASET = 'raw_ppg.csv'
TOPIC = 'topic_ppg'

df = pd.read_csv(DATASET,encoding='unicode_escape')


# iterate df for 10 rows and convert the row to json
for index, row in df.iterrows():
    row_json = row.to_dict()
    row_json["index"] = index
    data = json.dumps(row_json, default=str).encode('utf-8')
    producer.produce(topic=TOPIC, value=data)
    sleep(0.01)
    
producer.flush()